<a href="https://www.kaggle.com/rsizem2/tps-10-21-tensorflow-decision-forests-benchmarks?scriptVersionId=84829678" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Tensorflow Decision Forests

In this notebook we get benchmarks for the gradient boosting model provided with the [Tensorflow Decision Forests](https://www.tensorflow.org/decision_forests) library. For various input sizes (10k to 100k samples) we get AUC scores and training times for models with and without categorical features explicitly specified.

Personally, I don't think this library is ready to be used seriously for these competitions primarily because it runs so slowly (no GPU/TPU optimizations yet) and requires a linux environment (which I can only easily access through these notebooks). You can get similar results much easier and faster using one of the established gradient boosting frameworks like XGBoost, LightGBM, and CatBoost.


**Note:** This notebook will take several hours to run

In [1]:
# Global Variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_TREES = 1000
EARLY_STOP = 25

In [2]:
# Install TFDF library
!pip3 install -q tensorflow_decision_forests --upgrade

In [3]:
import numpy as np
import pandas as pd
import pyarrow
import warnings
import time
import gc
import os

# Hide warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Model and evaluation
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from collections import defaultdict

# Tensorflow
import tensorflow as tf
import tensorflow_decision_forests as tfdf
tf.random.set_seed(RANDOM_SEED)

# Preparing Data

1. Load original data
2. Downcast datatypes wherever possible
3. Split data into halves, use one half for estimation
4. Save data in pandas and tensorflow formats

In [4]:
%%time
# Load original training data
train = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")

# List for tracking categorical features
categorical_features = list()

# Downcast training data while keeping track of categorical variables
for col in train.columns:
    if train[col].dtype == "int64":
        train[col] = train[col].astype('int32')
        # ignore target column
        if col == "target": continue
        categorical_features.append(
            tfdf.keras.FeatureUsage(
                name = col, 
                semantic = tfdf.keras.FeatureSemantic.CATEGORICAL
            )
        )
    elif train[col].dtype == "float64":
        train[col] = pd.to_numeric(train[col], downcast ='float')

# Halve the data, we will get AUC estimates on a holdout set
train, holdout = train_test_split(
    train,
    train_size = 500000,
    stratify = train['target'],
    shuffle = True
)

# Save pandas dataframe for quick retrieval later
train.reset_index(drop = True, inplace = True)
holdout.reset_index(drop = True, inplace = True)
train.to_feather('train_500k.feather')
holdout.to_feather('holdout_full.feather')
del train; gc.collect()

# Create tensorflow data
holdout_tf = tfdf.keras.pd_dataframe_to_tf_dataset(
    holdout,
    label = 'target',
    in_place = True
)

# Save Train data
tf.data.experimental.save(holdout_tf, "holdout_tf")

# Garbage Collection (free up memory)
del holdout, holdout_tf; gc.collect()

CPU times: user 3min 12s, sys: 56 s, total: 4min 8s
Wall time: 4min 25s


0

# Helper Functions

We create a several function to perform various steps of the training and evaluation process, mostly to avoid having too many things loaded in memory at once.

In [5]:
def get_training_data(n_rows = 10000):
    
    assert 0 < n_rows < 500000
    train = pd.read_feather('train_500k.feather')
    
    train, test = train_test_split(
        train,
        train_size = n_rows,
        stratify = train['target'],
        shuffle = True
    )
    
    # Prepare Train Data
    train_df = tfdf.keras.pd_dataframe_to_tf_dataset(
        train,
        label = 'target',
        in_place = True
    )
    
    return train_df

In [6]:
def train_model(n_rows = 10000, categorical = False):

    train_df = get_training_data(n_rows)
    gc.collect()

    start = time.time()
    # Define model, using explicitly defined categoricals
    if categorical:
        model = tfdf.keras.GradientBoostedTreesModel(
            task = tfdf.keras.Task.CLASSIFICATION,
            num_trees = NUM_TREES,
            early_stopping_num_trees_look_ahead = EARLY_STOP,
            features = categorical_features,
            exclude_non_specified_features = False,
            verbose = 0
        )
    else:
        model = tfdf.keras.GradientBoostedTreesModel(
            task = tfdf.keras.Task.CLASSIFICATION,
            num_trees = NUM_TREES,
            early_stopping_num_trees_look_ahead = EARLY_STOP,
            verbose = 0
        )

    # Metric for validation 
    model.compile(
        metrics=[tf.metrics.AUC()]
    )

    # Training
    model.fit(train_df, verbose = 0)
    end = time.time()

    # Delete training data (free up memory)
    del train_df
    gc.collect()
    
    return model, round(end-start, 6)

In [7]:
def get_holdout_preds(model):
    
    holdout_df = tf.data.experimental.load("holdout_tf")

    preds = model.predict(holdout_df)[:,0]

    del holdout_df
    gc.collect()
    return preds

In [8]:
def get_holdout_score(y_preds):
    
    holdout = pd.read_feather('holdout_full.feather')
    y_true = holdout['target']
    
    return roc_auc_score(y_true, y_preds)

# Benchmarks

In [9]:
def get_benchmarks():
    
    data = defaultdict(list)
    
    for training_size in [10000, 20000, 30000, 40000, 50000, 75000]:

        # Train model, no specified categorical features
        model, training_time = train_model(n_rows = training_size)
        preds = get_holdout_preds(model)
        score = get_holdout_score(preds)
        print('All Numerical Features')
        print(f'Rows: {training_size}, Time: {round(training_time, 2)}')
        print(f'Validation Score: {round(score, 6)}\n')
        
        # save results
        data['size'].append(training_size)
        data['features'].append('numerical')
        data['time'].append(training_time)
        data['auc'].append(score)
        
        # free up memory
        del model
        gc.collect()
        
        # Train model, specify categorical features
        model, training_time = train_model(
            n_rows = training_size, 
            categorical = True
        )
        preds = get_holdout_preds(model)
        score = get_holdout_score(preds)
        print('Categorical Features')
        print(f'Rows: {training_size}, Time: {round(training_time, 2)}')
        print(f'Validation Score: {round(score, 6)}\n')
        
        # save results
        data['size'].append(training_size)
        data['features'].append('categorical')
        data['time'].append(training_time)
        data['auc'].append(score)
        
        # free up memory
        del model
        gc.collect()
        
    return pd.DataFrame(data)

In [10]:
# Output has been hidden
data = get_benchmarks()

All Numerical Features
Rows: 10000, Time: 66.59
Validation Score: 0.828674

Categorical Features
Rows: 10000, Time: 460.63
Validation Score: 0.827959

All Numerical Features
Rows: 20000, Time: 140.35
Validation Score: 0.837898

Categorical Features
Rows: 20000, Time: 1433.92
Validation Score: 0.836397

All Numerical Features
Rows: 30000, Time: 211.13
Validation Score: 0.841015

Categorical Features
Rows: 30000, Time: 2187.41
Validation Score: 0.84071

All Numerical Features
Rows: 40000, Time: 328.85
Validation Score: 0.843139

Categorical Features
Rows: 40000, Time: 3568.73
Validation Score: 0.841901

All Numerical Features
Rows: 50000, Time: 415.52
Validation Score: 0.844958

Categorical Features
Rows: 50000, Time: 6461.87
Validation Score: 0.843859

All Numerical Features
Rows: 75000, Time: 816.6
Validation Score: 0.847515

Categorical Features
Rows: 75000, Time: 6225.95
Validation Score: 0.846796



In [11]:
data

,size,features,time,auc
0,10000,numerical,66.592544,0.828674
1,10000,categorical,460.633493,0.827959
2,20000,numerical,140.350190,0.837898
3,20000,categorical,1433.921919,0.836397
4,30000,numerical,211.129573,0.841015
5,30000,categorical,2187.409756,0.840710
6,40000,numerical,328.848469,0.843139
7,40000,categorical,3568.732875,0.841901
8,50000,numerical,415.524760,0.844958
9,50000,categorical,6461.873198,0.843859


We see that explicitly specifying the categorical features significantly increases the training times but does not result in notably better models so we will not bother specifying categorical features for subsequent models.